In [13]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep, time
import cv2 as cv
import os
import random
import pyautogui

In [10]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def _init_(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img)

        return img

    def get_window_size(self):
        return (self.w, self.h)

In [11]:
class ImageProcessor:
    def _init_(self, img_size, cfg_file, weights_file):
        self.W, self.H = img_size
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = {}
        self.colors = {}
        self.car_counts = {}

        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()
            if self.classes[i] == 'car':
                self.car_counts[i] = 0

        self.colors = [
            (0, 255, 0),
            (0, 0, 255),
            (255, 0, 0),
            (255, 255, 0),
            (255, 0, 255),
            (0, 255, 255)
        ]

    def process_image(self, img):
        blob = cv.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)

        coordinates = self.get_coordinates(outputs, 0.5)

        self.draw_identified_objects(img, coordinates)

        return coordinates

    def get_coordinates(self, outputs, conf):
        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w // 2), int(y - h // 2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf - 0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})
        return coordinates

    def calculate_distance(self, obj1, obj2):
        center_x1 = obj1['x'] + obj1['w'] // 2
        center_y1 = obj1['y'] + obj1['h'] // 2
        center_x2 = obj2['x'] + obj2['w'] // 2
        center_y2 = obj2['y'] + obj2['h'] // 2
        distance = ((center_x1 - center_x2) * 2 + (center_y1 - center_y2) * 2) ** 0.5
        return distance

    def draw_identified_objects(self, img, coordinates):
        car_objects = [c for c in coordinates if c["class_name"] == "car"]
        handicap_objects = [c for c in coordinates if c["class_name"] == "handicap"]
        count_dict = self.car_counts.copy()

        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']

            color = self.colors[classID]

            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if coordinate in car_objects:
                for handicap in handicap_objects:
                    distance = self.calculate_distance(coordinate, handicap)
                    center_x = (coordinate['x'] + coordinate['w'] // 2 + handicap['x'] + handicap['w'] // 2) // 2
                    center_y = (coordinate['y'] + coordinate['h'] // 2 + handicap['y'] + handicap['h'] // 2) // 2
                    cv.putText(img, f'{distance:.1f}', (center_x, center_y), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            if coordinate in car_objects:
                count = count_dict[classID]
                cv.putText(img, str(count), (x + 10, y + 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                count_dict[classID] += 1

                center_x = x + w // 2
                center_y = y + h // 2
                cv.line(img, (center_x, y), (center_x, y + h), color, 1)  # Vertical line within the bounds
                cv.line(img, (x, center_y), (x + w, center_y), color, 1)  # Horizontal line within the bounds
                cv.putText(img, 'om', (center_x, center_y), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        cv.imshow('DETECTED OBJECTS', img)

In [12]:
import pyautogui
from time import sleep

sleep(4)

window_name = "SM-A236E"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)

# Set pyautogui to be faster
pyautogui.PAUSE = 0  # No delay between actions

TOP_LIMIT = 150  # Define the top limit in pixels

clicked_car_objects = set()  # Store coordinates of clicked car objects
clicked_handicap_objects = set()  # Store coordinates of clicked handicap objects

while True:
    ss = wincap.get_screenshot()
    
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break

    coordinates = improc.process_image(ss)
    car_objects = [c for c in coordinates if c["class_name"] == "car"]
    handicap_objects = [c for c in coordinates if c["class_name"] == "handicap"]

    new_car_objects = []
    
    for car in car_objects:
        car_center = (car['x'] + car['w'] // 2, car['y'] + car['h'] // 2)
        if car_center not in clicked_car_objects:
            new_car_objects.append(car)

    for car in new_car_objects:
        center_x = car['x'] + car['w'] // 2
        center_y = car['y'] + car['h'] // 2
        clicked_car_objects.add((center_x, center_y))

        # Click on the center point of the vertical line in the middle of the car object
        pyautogui.click(center_x, center_y)

    new_handicap_objects = []
    
    for handicap in handicap_objects:
        handicap_center = (handicap['x'] + handicap['w'] // 2, handicap['y'] + handicap['h'] // 2)
        if handicap_center not in clicked_handicap_objects:
            new_handicap_objects.append(handicap)

    for handicap in new_handicap_objects:
        center_x = handicap['x'] + handicap['w'] // 2
        center_y = handicap['y'] + handicap['h'] // 2
        clicked_handicap_objects.add((center_x, center_y))

        # Click on the center point of the vertical line in the middle of the handicap object
        pyautogui.click(center_x, center_y)

print('Finished.')

C:\Users\dizee\AppData\Local\Temp\ipykernel_32248\20239051.py:32: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


KeyboardInterrupt: 

In [ ]:
import pyautogui
from time import sleep

window_name = "SM-A236E"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)

screen_height = 560
border_pixels = 8
titlebar_pixels = 33
velocity_multiplier = 2

previous_coordinates = []
new_coordinates = []

clicked_car_objects = set()  # Store coordinates of clicked car objects

sleep(4)

# Set pyautogui to be faster
pyautogui.PAUSE = 0 # No delay between actions

def distance_to_point(fruit, point_x, point_y):
    center_x = fruit['x'] + fruit['w'] // 2
    center_y = fruit['y'] + fruit['h'] // 2
    return ((center_x - point_x) * 2 + (center_y - point_y) * 2) ** 0.5

def click_center(obj):
    center_x = obj['x']
    center_y = obj['y']
    pyautogui.click(center_x, center_y)

while True:
    ss = wincap.get_screenshot()
    previous_coordinates = improc.process_image(ss)
    ss = wincap.get_screenshot()
    new_coordinates = improc.process_image(ss)

    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break
    
    car_objects = [c for c in new_coordinates if c["class_name"] == "car"]

    for car in car_objects:
        car_center_x = car['x'] + car['w'] // 2
        car_center_y = car['y'] + car['h'] // 2
        if (car_center_x, car_center_y) not in clicked_car_objects:
            click_center({'x': car_center_x, 'y': car_center_y, 'w': 0, 'h': 0, 'class': None, 'class_name': 'car'})
            clicked_car_objects.add((car_center_x, car_center_y))

print('Finished.')